In [47]:
# Import necessary libraries
from __future__ import print_function
import torch.utils.data as data
from PIL import Image
import os
import os.path
import errno
import numpy as np
import torch
import codecs

In [48]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assume that we are on a CUDA machine, then this should print a CUDA device:

print(device)

cpu


In [49]:
class MNIST(data.Dataset):
    """`MNIST <http://yann.lecun.com/exdb/mnist/>`_ Dataset.
    Args:
        root (string): Root directory of dataset where ``processed/training.pt``
            and  ``processed/test.pt`` exist.
        train (bool, optional): If True, creates dataset from ``training.pt``,
            otherwise from ``test.pt``.
        download (bool, optional): If true, downloads the dataset from the internet and
            puts it in root directory. If dataset is already downloaded, it is not
            downloaded again.
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
    """
    urls = [
        'http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz',
        'http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz',
        'http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz',
        'http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz',
    ]
    raw_folder = 'raw'
    processed_folder = 'processed'
    training_file = 'training.pt'
    test_file = 'test.pt'
    classes = ['0 - zero', '1 - one', '2 - two', '3 - three', '4 - four',
               '5 - five', '6 - six', '7 - seven', '8 - eight', '9 - nine']
    class_to_idx = {_class: i for i, _class in enumerate(classes)}

    @property
    def targets(self):
        if self.train:
            return self.train_labels
        else:
            return self.test_labels

    def __init__(self, root, train=True, transform=None, target_transform=None, download=False):
        self.root = os.path.expanduser(root)
        self.transform = transform
        self.target_transform = target_transform
        self.train = train  # training set or test set

        if download:
            self.download()

        if not self._check_exists():
            raise RuntimeError('Dataset not found.' +
                               ' You can use download=True to download it')

        if self.train:
            self.train_data, self.train_labels = torch.load(
                os.path.join(self.root, self.processed_folder, self.training_file))
        else:
            self.test_data, self.test_labels = torch.load(
                os.path.join(self.root, self.processed_folder, self.test_file))

    def __getitem__(self, index):
        """
        Args:
            index (int): Index
        Returns:
            tuple: (image, target) where target is index of the target class.
        """
        if self.train:
            img, target = self.train_data[index], self.train_labels[index]
        else:
            img, target = self.test_data[index], self.test_labels[index]

        # doing this so that it is consistent with all other datasets
        # to return a PIL Image
        img = Image.fromarray(img.numpy(), mode='L')

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target

    def __len__(self):
        if self.train:
            return len(self.train_data)
        else:
            return len(self.test_data)

    def _check_exists(self):
        return os.path.exists(os.path.join(self.root, self.processed_folder, self.training_file)) and \
            os.path.exists(os.path.join(self.root, self.processed_folder, self.test_file))

    def download(self):
        """Download the MNIST data if it doesn't exist in processed_folder already."""
        from six.moves import urllib
        import gzip

        if self._check_exists():
            return

        # download files
        try:
            os.makedirs(os.path.join(self.root, self.raw_folder))
            os.makedirs(os.path.join(self.root, self.processed_folder))
        except OSError as e:
            if e.errno == errno.EEXIST:
                pass
            else:
                raise

        for url in self.urls:
            print('Downloading ' + url)
            data = urllib.request.urlopen(url)
            filename = url.rpartition('/')[2]
            file_path = os.path.join(self.root, self.raw_folder, filename)
            with open(file_path, 'wb') as f:
                f.write(data.read())
            with open(file_path.replace('.gz', ''), 'wb') as out_f, \
                    gzip.GzipFile(file_path) as zip_f:
                out_f.write(zip_f.read())
            os.unlink(file_path)

        # process and save as torch files
        print('Processing...')

        training_set = (
            read_image_file(os.path.join(self.root, self.raw_folder, 'train-images-idx3-ubyte')),
            read_label_file(os.path.join(self.root, self.raw_folder, 'train-labels-idx1-ubyte'))
        )
        test_set = (
            read_image_file(os.path.join(self.root, self.raw_folder, 't10k-images-idx3-ubyte')),
            read_label_file(os.path.join(self.root, self.raw_folder, 't10k-labels-idx1-ubyte'))
        )
        with open(os.path.join(self.root, self.processed_folder, self.training_file), 'wb') as f:
            torch.save(training_set, f)
        with open(os.path.join(self.root, self.processed_folder, self.test_file), 'wb') as f:
            torch.save(test_set, f)

        print('Done!')

    def __repr__(self):
        fmt_str = 'Dataset ' + self.__class__.__name__ + '\n'
        fmt_str += '    Number of datapoints: {}\n'.format(self.__len__())
        tmp = 'train' if self.train is True else 'test'
        fmt_str += '    Split: {}\n'.format(tmp)
        fmt_str += '    Root Location: {}\n'.format(self.root)
        tmp = '    Transforms (if any): '
        fmt_str += '{0}{1}\n'.format(tmp, self.transform.__repr__().replace('\n', '\n' + ' ' * len(tmp)))
        tmp = '    Target Transforms (if any): '
        fmt_str += '{0}{1}'.format(tmp, self.target_transform.__repr__().replace('\n', '\n' + ' ' * len(tmp)))
        return fmt_str


class FashionMNIST(MNIST):
    """`Fashion-MNIST <https://github.com/zalandoresearch/fashion-mnist>`_ Dataset.
    Args:
        root (string): Root directory of dataset where ``processed/training.pt``
            and  ``processed/test.pt`` exist.
        train (bool, optional): If True, creates dataset from ``training.pt``,
            otherwise from ``test.pt``.
        download (bool, optional): If true, downloads the dataset from the internet and
            puts it in root directory. If dataset is already downloaded, it is not
            downloaded again.
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
    """
    urls = [
        'http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz',
        'http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz',
        'http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-images-idx3-ubyte.gz',
        'http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz',
    ]
    classes = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal',
               'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
    class_to_idx = {_class: i for i, _class in enumerate(classes)}

def get_int(b):
    return int(codecs.encode(b, 'hex'), 16)


def read_label_file(path):
    with open(path, 'rb') as f:
        data = f.read()
        assert get_int(data[:4]) == 2049
        length = get_int(data[4:8])
        parsed = np.frombuffer(data, dtype=np.uint8, offset=8)
        return torch.from_numpy(parsed).view(length).long()


def read_image_file(path):
    with open(path, 'rb') as f:
        data = f.read()
        assert get_int(data[:4]) == 2051
        length = get_int(data[4:8])
        num_rows = get_int(data[8:12])
        num_cols = get_int(data[12:16])
        images = []
        parsed = np.frombuffer(data, dtype=np.uint8, offset=16)
    return torch.from_numpy(parsed).view(length, num_rows, num_cols)

In [50]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.autograd import Variable
train_dataset=FashionMNIST(root='./data',train=True,transform=transforms.ToTensor(),download=True)
test_dataset=FashionMNIST(root='./data',train=False,transform=transforms.ToTensor(),download=True)
batch_size=100
n_iters=18000
num_epochs=n_iters/(len(train_dataset)/batch_size)
num_epochs=int(num_epochs)
# load data
train_loader=torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_loader=torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)

In [51]:
class CNNModule(nn.Module):
    def __init__(self):
        super (CNNModule,self).__init__()

        # Convolution 1
        
        self.cnn1 = nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5,stride=1,padding=2)
        # nn.Dropout2d(p=0.5, inplace=False)
        self.relu1=nn.ReLU()                  # SELU()
        self.conv1_bn = nn.BatchNorm2d(16)
        nn.init.xavier_uniform_(self.cnn1.weight)
        
        #Max pool 1
        self.maxpool1=nn.MaxPool2d(kernel_size=2)
        
        # Convolution 2
        
        self.cnn2=nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5,stride=1,padding=2)
        # nn.Dropout2d(p=0.5, inplace=False)
        self.relu2=nn.ReLU()                 # SELU()
        self.conv2_bn = nn.BatchNorm2d(32)
        nn.init.xavier_uniform_(self.cnn2.weight)
        
        # Max pool 2
        self.maxpool2=nn.MaxPool2d(kernel_size=2)
        
        # Convolution 3
        
        self.cnn3 = nn.Conv2d(in_channels= 32, out_channels= 64, kernel_size=5,stride=1, padding=2)
        self.relu3 = nn.ReLU()
        self.conv3_bn = nn.BatchNorm2d(64)
        nn.init.xavier_uniform_(self.cnn3.weight)
        
        # Max pool 3
        self.maxPool3 = nn.MaxPool2d(kernel_size=2)

        
        # Fully connected network
        
        
        """
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(576, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 10)
        )
        
        """
        # Fully connected 1
        self.fcl=nn.Linear(576,10)          # 2 layers= 32*7*7
        
    def forward(self,x):
        out=self.cnn1(x)
        out=self.relu1(out)
        out = self.conv1_bn(out)
        
        out=self.maxpool1(out)
        
        out=self.cnn2(out)
        out=self.relu2(out)
        out = self.conv2_bn(out)
        out=self.maxpool2(out)
        
        # out size (100,32,7,7)
        # feed (100,32*7*7) to linear fn
        
        # out=self.dropout(out)
        
        out= self.cnn3(out)
        out = self.relu3(out)
        out = self.conv3_bn(out)
        out = self.maxPool3(out)
        
         
        
        out=out.view(out.size(0),-1)
        
        out=self.fcl(out)
        # out = self.classifier(out)
        
        
        return out

In [ ]:
model=CNNModule()
criterion=nn.CrossEntropyLoss()

learning_rate=0.03  #0.03
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)

In [ ]:
iter = 0
for epoch in range(num_epochs):                        # 5 fold times
    for i, (images,labels) in enumerate(train_loader): # all 60000 images
        #load images as variables
        #images = Variable(images.view(-1,28*28))
        ##### using images = images.reshape(-1,28*28)
        #labels = Variable(labels)
        #clear gradients wrt parameters
        images=Variable(images)
        labels=Variable(labels)
        optimizer.zero_grad()
        #forward pass to get output/logits
        outputs = model(images)
        #calculate loss--> softmax to cross entropy loss
        loss = criterion(outputs , labels)
        #getting gradients wrt parameters
        loss.backward()
        #updating parameters
        optimizer.step()
        
        iter+=1
        
        #new for logistic regression
        if iter%500==0:
            #calculate accuracy
            correct = 0
            total = 0
            # iterate through test dataset
            for images,labels in test_loader:
                #load images to torch Variable
                images = Variable(images)
                #forward pass only to get output
                outputs = model(images)
                #get predictions from the max value
                _,predicted = torch.max(outputs.data,1)
                #total num of labels
                total += labels.size(0)
                #total correct predictions
                correct += (predicted == labels).sum()
                
            accuracy = 100*(float(correct)/total)
            
            #print loss
            print ("Iterations: {}. Loss: {}. Accuracy: {}".format(iter,loss.data[0], accuracy))

/home/omkar/Installed/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Iterations: 500. Loss: 0.2875724136829376. Accuracy: 86.76
Iterations: 1000. Loss: 0.287662535905838. Accuracy: 88.53999999999999
Iterations: 1500. Loss: 0.186595618724823. Accuracy: 89.09
Iterations: 2000. Loss: 0.39148885011672974. Accuracy: 89.07000000000001
Iterations: 2500. Loss: 0.1426483690738678. Accuracy: 89.63
Iterations: 3000. Loss: 0.22725093364715576. Accuracy: 90.41
Iterations: 3500. Loss: 0.2617591917514801. Accuracy: 90.77
Iterations: 4000. Loss: 0.15027019381523132. Accuracy: 90.59
Iterations: 4500. Loss: 0.1257735937833786. Accuracy: 90.27
Iterations: 5000. Loss: 0.1373767852783203. Accuracy: 90.64
Iterations: 5500. Loss: 0.1377425640821457. Accuracy: 90.60000000000001
Iterations: 6000. Loss: 0.12841109931468964. Accuracy: 90.99000000000001
Iterations: 6500. Loss: 0.1065763533115387. Accuracy: 90.51
Iterations: 7000. Loss: 0.18179963529109955. Accuracy: 90.47
Iterations: 7500. Loss: 0.09633537381887436. Accuracy: 90.12
Iterations: 8000. Loss: 0.10868176817893982. Accu